### Optical Flow 
optical flow refers to the apparent motion of the object due to the relative motion of the object or the observer, it creats 2D vectors that determines the change in the position of the pixels from one frame to another, it uses the change in the intensity of the pixel to calculate the motion of the pixel, thus it assumes that the intensity of the pixel remain constant over a very short time interval as it moves across the image plane. 

f I(x,y,t) is the intensity of a pixel at coordinates (x,y) at time t, and this pixel moves to (x+dx,y+dy) at time t+dt, then:

![My Image](Images/op.png)


The given equation has two unknowns U and V, that can be determined by different methods. Optical flow algorithms can be broadly categorized into:
* Sparse Optical Flow (These algorithm track the motion of the Previous_point like corners and edges only)
* Dense Optical Flow (These algorithm track the motion of all the pixels in the image)

### Lucas Kanade Method
It is a sparse optical flow algorithm which assumes that the optical flow (u,v) is approximately constant within a small local neighborhood around the pixel under consideration. thus the value of U and V would be the same for a neighborhood of pixels, it first uses the She-Tomasi corner detection or GoodPrevious_pointToTrack to determine important Previous_point then take a 3×3 window (9 pixels) around a central pixel, then all 9 pixels in that window are assumed to have the same (u,v) velocity vector. Thus we can then calculate the value of U and V using 9 pixels.

In [1]:
import numpy as np
import cv2 as cv

In [2]:
image_path = "Images/recording.mp4"
cap = cv.VideoCapture(image_path)
ret, old_frame = cap.read() # Taking the first frame of the image
old_gray_frame = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

In [3]:
# Getting the Previous_point of the old frame (Both corners and Keypoints can be used as PreviousPoints in Optical Flow)
Previous_point = cv.goodFeaturesToTrack(old_gray_frame, 500, 0.3, 10) 
orb = cv.ORB_create(200)
kp1 = orb.detect(old_gray_frame, None)
Previous_point = np.array([kp.pt for kp in kp1], dtype=np.float32).reshape(-1, 1, 2) # Turining Keypoints Object to points list
# Creating mask for drawing optic flow vectors
mask = np.zeros_like(old_frame)

In [ ]:
while True:
    ret, frame = cap.read() 
    if not ret: 
        print("Frame not found")
        break 
    
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Applying Lucas Kanade 
    nextPoints, status, error = cv.calcOpticalFlowPyrLK(old_gray_frame, gray_frame, Previous_point, None) # (old_frame, new_frame, old_points)
    # status is set to 0 for the points with no flow found else its 1
    
    # Accessing old and new points whose flow is found 
    if nextPoints is not None:
        good_new = nextPoints[status == 1]
        good_old = Previous_point[status == 1]

    # Drawing Flow lines
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), [0,255,0], 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, [0,255,0], -1)
    
    # Displaying the image    
    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    if cv.waitKey(30) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break
    # Updating the oldframe
    old_gray_frame = gray_frame.copy()
    Previous_point = good_new.reshape(-1,1,2)
